***Thêm thư viện cần thiết***

In [22]:
import json
import re
import numpy as np
import pandas as pd
import scrapy

ImportError: cannot import name 'to_unicode' from 'scrapy.utils.python' (C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scrapy\utils\python.py)

In [16]:
!cd ../data

c:\Users\ASUS\Desktop\NMKHDL\src\data


***Tạo thư mục làm việc của Scrapy***

In [17]:
# !scrapy startproject air_quality #Tạo thư mục làm việc air_quality

In [18]:
!cd air_quality #Di chuyển vào thư mục làm việc 

The system cannot find the path specified.


***Bắt đầu thu thập dữ liệu***

In [19]:
from air_quality.items import AirQualityItem

class collect_air_data(scrapy.Spider): #Khởi tạo class dùng để thu thập
    name = 'air_datas' #Tên spider
    
    data = AirQualityItem() #spider Field để chuyển đổi sang json
    
    #Bắt đầu request
    def start_requests(self):
        start = 'https://www.iqair.com/vi/world-most-polluted-cities?sort=-rank&page=1&perPage=50' #Url bắt đầu

        urls = [] #Danh sách url cần lấy dữ liệu
        
        for i in range(1, 131): #Có tổng cộng 130 url, lặp qua để thêm vào danh sách urls
            url = start.replace('1', str(i))
            urls.append(url)
            
        for url in urls: #Thực hiện request với từng url
            yield scrapy.Request(url = url, callback = self.parse)
    
    #Lấy dữ liệu cần thiết
    def parse(self, response):
        #Tìm thẻ tbody chứa bảng 50 thành phố
        res = response.css('tbody[role = "rowgroup"]') 
        
        #Thêm từng thành phố vào list_rank, mỗi thành phố được định nghĩa trong thẻ tr
        list_rank = np.array(res.css('tr')) 
        
        #Với mỗi thành phố trích xuất dữ liệu cần thiết, những dữ liệu này đều nằm trong thẻ td
        for i in list_rank:
            city = i.css('td')
            
            self.data['Rank'] = city[0].css('td::text').get() #Hạng
            self.data['City'] = city[1].css('div::text').get() #Tên thành phố
            
            self.data['Year_2021'] = city[2].css('span::text').get() #Chỉ số năm 2021
            
            self.data['Jan'] = city[3].css('span::text').get() #Chỉ số tháng 1
            self.data['Feb'] = city[4].css('span::text').get() #Chỉ số tháng 2
            self.data['Mar'] = city[5].css('span::text').get() #Chỉ số tháng 3
            self.data['Apr'] = city[6].css('span::text').get() #Chỉ số tháng 4
            self.data['May'] = city[7].css('span::text').get() #Chỉ số tháng 5
            self.data['Jun'] = city[8].css('span::text').get() #Chỉ số tháng 6
            self.data['Jul'] = city[9].css('span::text').get() #Chỉ số tháng 7
            self.data['Aug'] = city[10].css('span::text').get() #Chỉ số tháng 8
            self.data['Sep'] = city[11].css('span::text').get() #Chỉ số tháng 9
            self.data['Oct'] = city[12].css('span::text').get() #Chỉ số tháng 10
            self.data['Nov'] = city[13].css('span::text').get() #Chỉ số tháng 11
            self.data['Dec'] = city[14].css('span::text').get() #Chỉ số tháng 12
            
            self.data['Year_2020'] = city[15].css('span::text').get() #Chỉ số năm 2020
            self.data['Year_2019'] = city[16].css('span::text').get() #Chỉ số năm 2019
            self.data['Year_2018'] = city[17].css('span::text').get() #Chỉ số năm 2018
            self.data['Year_2017'] = city[18].css('span::text').get() #Chỉ số năm 2017
                
            yield self.data #Trả về spider Field data

ModuleNotFoundError: No module named 'air_quality.items'

Vì thư viện scrapy không thể thực hiện lấy dữ liệu trực tiếp từ jupyter notebook, nên cần thực hiện tạo file collect_air_data.py trong đường dẫn **air_quality/air_qualiy/spiders/collect_air_data.py**. File collect_air_data.py sẽ chứa code của cell bên trên

In [20]:
!scrapy crawl air_datas -o dataset/air_datas.json #Lấy dữ liệu và lưu vào file air_datas.json trong thư mục dataset

'scrapy' is not recognized as an internal or external command,
operable program or batch file.


***Đọc dataframe từ file json***

In [ ]:
#Tao dataframe df_test để đọc air_data.json
df_test = pd.read_json('dataset/air_datas.json', encoding='utf-8-sig')

df_test 

,Rank,City,Year_2021,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year_2020,Year_2019,Year_2018,Year_2017
0,1,"Bhiwadi, India",106.2,145.8,129.8,120.2,125.7,86.5,95.9,55.6,55.4,37.1,91.1,188.6,136.6,95.5,83.4,125.4,-
1,2,"Ghaziabad, India",102.0,199.9,172.2,97.8,86.3,52.9,47.2,35.3,37.6,30.8,89.7,218.3,163,106.6,110.2,135.2,144.6
2,3,"Hotan, China",101.5,-,-,158,91.1,167.4,57.4,70.9,93.2,79.3,126.1,111.5,62.6,110.2,110.1,116,91.9
3,4,"Delhi, India",96.4,183.7,142.2,80.5,72.9,47.4,47.1,35.6,36.9,30.2,73.7,224.1,186.4,84.1,98.6,113.5,108.2
4,5,"Jaunpur, India",95.3,182.2,143.5,91,70,51.1,40.7,33.5,34.2,36.8,75.7,196,195.7,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6470,6296,"Alandroal, Portugal",4.9,3.9,4.8,5.9,4.7,3.5,5.2,5.8,5.8,4.4,-,4.5,4.5,4.8,5.3,7.1,13.3
6471,6297,"Björklinge, Sweden",4.9,4,4.5,5.7,2.3,4.6,7.7,6.5,3.5,3.8,5.9,7,5.1,-,-,-,-
6472,6298,"Bothwell, United Kingdom",4.9,3.6,24,4.4,6.7,4.4,5.2,5.5,4.9,6.1,3.2,3.7,5.3,-,-,-,-
6473,6299,"Currie, United Kingdom",4.9,-,-,4.9,6,4.6,5.1,5.9,4.6,5.9,3.2,3,4.9,-,-,-,-
